In [5]:
cd Desktop/Globsyn/dataset/

C:\Users\Pravash Ranjan\Desktop\Globsyn\dataset


In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import feature_selection
from sklearn import tree
from sklearn import utils
from sklearn import naive_bayes
from sklearn import feature_selection
from sklearn import ensemble
import re


In [6]:
df=pd.read_csv("Interview.csv")
df.drop(["Unnamed: 23","Unnamed: 24","Unnamed: 25","Unnamed: 26","Unnamed: 27"],axis=1,inplace=True)
df.columns=["date","client","industry","location","position","skills","inter_type","ID","gender","curr_location","job_location",
           "venue","native_location","permission","hope","three_hours","alt_number","resume","clear_about_venue","letter","expected",
           "observed","married"]

In [7]:
df.date=df.date.str.replace(" ","")
df.date=df.date.str.replace(r'\d\d\d\d&\d\d\.\d\d\w\w',"2016")
df.date=df.date=df.date.str.replace("/",".")
df.date=df.date.str.replace(r'\WApr\W',".04.")
df.date=df.date.str.replace('Apr',".04.")
df.date=df.date.str.replace(r'\d\d\d\d&\d\.\d\d\w\w',"2016")
df.date=df.date.str.replace('-',".")
df.date=df.date.str.replace(r'\W16',".2016")
df.date=df.date.str.replace(r'\W15',".2015")
df.drop(df[df.date.isnull()].index,inplace=True)
import datetime
def conv(s):
    ar=str(s).split(".")
    try:
        d=datetime.date(int(ar[2]),int(ar[1]),int(ar[0]))
    except Exception:
        print(len(ar),s)
    return (d)
df["new_date"]=df.date.apply(conv)
df["new_date"]=pd.to_datetime(df.new_date)
df.drop(df[df.expected.isnull()].index,inplace=True)
df.drop("date",axis=1,inplace=True)
df.client.replace({"Standard Chartered Bank Chennai":"Standard Chartered Bank","Aon hewitt Gurgaon":"Aon Hewitt","Hewitt":"Aon Hewitt"},inplace=True)
df.industry.replace({"IT Products and Services":"IT","IT Services":"IT"},inplace=True)
df.location.replace({"CHENNAI":"Chennai","chennai":"Chennai","Gurgaonr":"Gurgaon","- Cochin- ":"Cochin","chennai ":"Chennai"},inplace=True)
df=df[df.new_date<'2019-01-01']
df.drop(df[(df.hope.isnull()) & (df.three_hours.isnull()) & (df.alt_number.isnull()) & (df.resume.isnull()) & (df.clear_about_venue.isnull())& (df.letter.isnull())].index,inplace=True)
df.observed.replace({"yes":"Yes","yes ":"Yes","no":"No","No ":"No","NO":"No","no ":"No"},inplace=True)
df.expected.replace({"yes":"Yes","11:00 AM":"Yes","10.30 Am":"Yes"},inplace=True)
df.letter.replace({"Havent Checked":"No","Need To Check":"No","Not sure":"No","Yet to Check":"No","Not Sure":"No","Not yet":"No",
                  "no":"No","na":"No","yes":"Yes","Na":"No"},inplace=True)
df.clear_about_venue.replace({"No- I need to check":"No","na":"No","yes":"Yes","Na":"No","no":"No"},inplace=True)
df.resume.replace({"No- will take it soon":"No","Not yet":"No","na":"No","yes":"Yes","Na":"No","Not Yet":"No"},inplace=True)
df.alt_number.replace({"No I have only thi number":"No","na":"No","yes":"Yes","Na":"No"},inplace=True)
df.three_hours.replace({"No Dont":"No","Na":"No","yes":"Yes"},inplace=True)
df.hope.replace({"Na":"No","yes":"Yes","Not Sure":"No","cant Say":"No","Not sure":"No"},inplace=True)
df.permission.replace({"Not yet":"No","Yet to confirm":"No","yes":"Yes","Na":"No"},inplace=True)
for i,s in df.iterrows():
    if (df["expected"][i]=="Uncertain"):
        df["expected"][i]=df["observed"][i]
df.permission.replace({"Yes":1,"No":0},inplace=True)
df.hope.replace({"Yes":1,"No":0},inplace=True)
df.three_hours.replace({"Yes":1,"No":0},inplace=True)
df.alt_number.replace({"Yes":1,"No":0},inplace=True)
df.resume.replace({"Yes":1,"No":0},inplace=True)
df.clear_about_venue.replace({"Yes":1,"No":0},inplace=True)
df.letter.replace({"Yes":1,"No":0},inplace=True)
df.expected.replace({"Yes":1,"No":0},inplace=True)
df.observed.replace({"Yes":1,"No":0},inplace=True)
df.married.replace({"Married":1,"Single":0},inplace=True)
df.drop(df[(df.permission.isnull()) | (df.hope.isnull()) | (df.three_hours.isnull()) | 
         (df.alt_number.isnull()) | (df.resume.isnull()) | (df.clear_about_venue.isnull()) |
        (df.letter.isnull()) | (df.expected.isnull()) | (df.observed.isnull())].index,inplace=True)
df.inter_type.replace({"Scheduled Walkin":"Scheduled","Scheduled ":"Scheduled","Scheduled Walk In":"Scheduled","Walkin ":"Walkin"},inplace=True)


In [8]:
loc=df.curr_location
ind=df.industry
cl=df.client
jbl=df.job_location
mar=df.married
X=df.drop(["ID","skills","new_date","native_location","curr_location","industry","client","job_location"],axis=1)

In [9]:
pos = pd.get_dummies(X['position'],prefix='pos',drop_first=True)
X = pd.concat([X,pos],axis=1)
X.drop("position",axis=1,inplace=True)
X.gender.replace({"Male":1,"Female":0},inplace=True)
X.inter_type.replace({"Scheduled":1,"Walkin":0},inplace=True)
typ=pd.get_dummies(X.venue,prefix="ven",drop_first=True)
X=pd.concat([X,typ],axis=1)
X.drop("venue",axis=1,inplace=True)
loc.replace({"chennai":"Chennai","CHENNAI":"Chennai","chennai ":"Chennai","- Cochin- ":"Cochin"},inplace=True)
jbl.replace({"- Cochin- ":"Cochin"},inplace=True)
dum_loc=pd.get_dummies(loc,prefix="loc",drop_first=True)
dum_ind=pd.get_dummies(ind,prefix="ind",drop_first=True)
dum_jbl=pd.get_dummies(jbl,prefix="jbl",drop_first=True)
X=pd.concat([X,dum_loc,dum_ind,dum_jbl],axis=1)
X.drop("location",axis=1,inplace=True)

In [12]:
y=X.observed
X.drop("observed",axis=1,inplace=True)

Precision with  1 features 0.7441860465116279
Precision with  2 features 0.7458379578246392
Precision with  6 features 0.7441860465116279
Precision with  6 features 0.7441860465116279
Precision with  6 features 0.7441860465116279
Precision with  6 features 0.7441860465116279
Precision with  16 features 0.7458379578246392
Precision with  16 features 0.7458379578246392
Precision with  16 features 0.7458379578246392

In [62]:
def trying():
    lr=LogisticRegression()
    #nb_G=naive_bayes.GaussianNB()
    nb_B=naive_bayes.BernoulliNB()
    nb_M=naive_bayes.MultinomialNB()
    rf=ensemble.RandomForestClassifier()
    m=[rf]
    s=["Random Forest"]
    for j in range(0,len(m)):
        print(s[j])
        model=m[j]
        for i in range(1,10):
            rfecv=feature_selection.RFECV(estimator=model,min_features_to_select=i,cv=5,scoring="precision")
            rfecv.fit(X,y)
            cols=X.columns[rfecv.get_support()]
            X_=X[cols]
            rfecv.fit(X_,y)
            
            #model.fit(X_,y)
            print("Precision with ",len(cols),"features",metrics.precision_score(y,rfecv.predict(X_)))

In [63]:
trying()

Random Forest
Precision with  8 features 0.7441860465116279
Precision with  2 features 0.7441860465116279
Precision with  23 features 0.755079006772009
Precision with  9 features 0.7436464088397791
Precision with  16 features 0.7600459242250287
Precision with  17 features 0.7447280799112098
Precision with  14 features 0.7548240635641317
Precision with  19 features 0.7565415244596132
Precision with  16 features 0.7591324200913242
